In [4]:
import os
import numpy as np
from nibabel.testing import data_path
import nibabel as nib
import matplotlib.pyplot as plt
import pandas as pd

from nonlocalunet.model.models import NonLocalUnet
from nonlocalunet.train.Metrics import dr_score, mhd_score

import torch
from torch.utils.data import Dataset, DataLoader, random_split
from torch import nn
import torch.nn.functional as F
import torchvision
from torchvision import transforms
import pytorch_lightning as pl

from liverfiles.loader import Ds

import random
from pytorch_lightning.loggers import TensorBoardLogger

# Data

In [5]:
pathes = []
for (dirpath, dirnames, filenames) in os.walk('../input/liver-tumor-segmentation'):
    if dirpath == '../input/liver-tumor-segmentation/segmentations':
        continue
    pathes.extend([os.path.join(dirpath, i)for i in filenames] )
for (dirpath, dirnames, filenames) in os.walk('../input/liver-tumor-segmentation-part-2'):
    if dirpath == '../input/liver-tumor-segmentation/segmentations':
        continue
    pathes.extend([os.path.join(dirpath, i)for i in filenames] )
df = pd.DataFrame({'path': pathes})
df['id'] = df.path.str.split('/').str[-1].str[7:-4]
df.id = df.id.astype('int16')
df = df.sort_values(by='id').reset_index(drop=True)

AttributeError: Can only use .str accessor with string values!

# Dataloader

In [6]:
shape = (32, 32, 32)

train_ds = Ds(df[:100], shape, None)
test_ds = Ds(df[100:], shape, None)
train_dl = DataLoader(train_ds,
                      batch_size=5,
                      shuffle=True,
                      num_workers=0,
                      pin_memory=False,
                      drop_last=True)
test_dl = DataLoader(test_ds,
                     batch_size=1,
                     shuffle=False,
                     num_workers=1,
                     pin_memory=False,
                     drop_last=False)
#test1, test2 = next(iter(train_dl))

ValueError: num_samples should be a positive integer value, but got num_samples=0

In [ ]:
a, b = next(iter(train_dl))
c = 0

In [ ]:
example = torchvision.utils.make_grid(torch.transpose(a[c], 0,1), nrow=6)
plt.figure(figsize=(20,20))
plt.imshow(example.permute(1,2,0))

In [ ]:
example = torchvision.utils.make_grid(torch.transpose(b[c], 0,1)[:,0:1,:,:].float(), nrow=6)
plt.figure(figsize=(20,20))
plt.imshow(example.permute(1,2,0))
c+=1

In [ ]:
class Liver(pl.LightningModule):
    def __init__(self, model, loss):
        super().__init__()
        self.model = model
        self.dice = DiceLoss()
        self.bce = nn.BCELoss()
        
    def forward(self, x):
        return self.model(x.float())
    
    def training_step(self, batch, batch_idx):
        x, y = batch
        preds = self.model(x.float())
        bce_loss = self.bce(preds, y.float())
        dice_loss = self.dice(preds, y.float())
        loss = bce_loss + 0.5*dice_loss
        self.log('train_bce', bce_loss)
        self.log('train_dice', dice_loss)
        self.log('train_loss', loss)
        return loss
    
    def validation_step(self, batch, batch_idx):
        x, y = batch
        preds = self.model(x.float())
        bce_loss = self.bce(preds, y.float())
        dice_loss = self.dice(preds, y.float())
        loss = bce_loss + 0.5*dice_loss
        self.log('test_bce', bce_loss)
        self.log('test_dice', dice_loss)
        self.log('test_loss', loss)
        return loss

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
        return optimizer

In [ ]:
model = NonLocalUnet(1, 2)
logger = TensorBoardLogger('tb_logs', name='my_model')
ligth = Liver(model, DiceLoss())
trainer = pl.Trainer(gpus=1, logger=logger)
trainer.fit(ligth, train_dl, test_dl)

In [ ]:
res = ligth(a.float())

In [ ]:
plt.imshow(b[2,0,31].detach())

Problem: 49